# Train CIFAR10 with PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

from models import *
from utils import progress_bar
from statistics import mean
best_acc=0
from tqdm import tqdm

In [2]:
# Training
def train():
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    batch_size=128
    counter=0
    total_iterations=len(trainloader)*batch_size 
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        counter+=1
        if counter %100==0:
            print('Iterations: {} | Train Loss: {} | Train Acc: {}'.format(str(counter*batch_size)+'/'+str(total_iterations),
                                                                           str(train_loss/(batch_idx+1)),
                                                                            str(100.*correct/total)))
    print('Iterations: {} | Train Loss: {} | Train Acc: {}'.format(str(counter*batch_size)+'/'+str(total_iterations),
                                                                           str(train_loss/(batch_idx+1)),
                                                                            str(100.*correct/total)))
#         progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                      % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return np.log10(train_loss),100.*correct/total



def test():
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    batch_size=100
    counter=0
    total_iterations=len(testloader)*batch_size
    global best_acc
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            counter+=1
            if counter%100==0:
                print('Iterations: {} | Test Loss: {} | Test Acc: {}'.format(str(counter*batch_size)+'/'+str(total_iterations),
                                                                             str(test_loss/(batch_idx+1)),                                                                          str(100.*correct/total)))

    # Save checkpoint.
    cur_acc = 100.*correct/total
    if cur_acc > best_acc:
        best_acc = cur_acc
    print('\nBest Testing Accuracy: {}'.format(str(best_acc)))
    return np.log10(test_loss),100.*correct/total
        
        
        
        

In [3]:
def choose_optim(opt_type):
    if opt_type=='SGD':
        optimizer=optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
    else:
        optimizer=optim.Adam(net.parameters(),lr=lr)
    return optimizer

# The function to show an image.
def imshow(img):
    img = img / 2 + 0.5     # Unnormalize.
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def get_class_accuracy(testloader):
    # Get test accuracy for each class.
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(100):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    accu_list=[]
    for i in range(10):
        val=100 * class_correct[i] / class_total[i]
        accu_list.append(val)
#         print('Accuracy of %5s : %2d %%' % (classes[i], val))
        
    return accu_list


def plot(train_loss_ls,test_loss_ls,train_accu_ls,test_accu_ls):
    fig,ax=plt.subplots(figsize=(8,4))
    ax1=plt.subplot(121)
    ax1.plot(np.arange(1,end_epoch+1,1),train_loss_ls)
    ax1.plot(np.arange(1,end_epoch+1,1),test_loss_ls)
    ax1.set_xlabel('num of epochs')
    ax1.set_ylabel('loss')
    # ax1.set_ylim((1,5))
    ax1.set_title('Log loss')
    ax1.legend(['training loss','testing loss'])

    ax2=plt.subplot(122)
    ax2.plot(np.arange(1,end_epoch+1,1),train_accu_ls)
    ax2.plot(np.arange(1,end_epoch+1,1),test_accu_ls)
    ax2.set_xlabel('num of epochs')
    ax2.set_ylabel('accuracy in %')
    ax2.set_title('accuracy')
    ax2.legend(['training accuracy','testing accuracy'])

    plt.show()

In [4]:
def list_update(list1, list2):
    return [a if (a>b) else b for a,b in zip(list1, list2) ]


In [5]:

lr=0.1
resume=False

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Begin Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
print('==> End Preparing data..')


==> Begin Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> End Preparing data..


In [6]:
model_list=[RegNetX_200MF().cuda()]  


In [7]:
# Model
print('==> Begin Building model..')
'''
net = VGG('VGG19')
net = ResNet18()
net = PreActResNet18()
net = GoogLeNet()
net = DenseNet121()
net = ResNeXt29_2x64d()
net = MobileNet()
net = MobileNetV2()
net = DPN92()
net = ShuffleNetG2()
net = SENet18()
net = ShuffleNetV2(1)
net = EfficientNetB0()
net = RegNetX_200MF()
net=PigNet3()
net=PigNet5()
net=PigNet7()
net = net.to(device)
'''
# if device == 'cuda':
#     net = torch.nn.DataParallel(net)
#     cudnn.benchmark = True
  
if resume:
#     Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']


# model_list=[PigNet3().cuda(),ResNet18().cuda(),PigNet5().cuda()] 
model_list2=[VGG('VGG11').cuda(),VGG2('VGG11').cuda(),VGG3('VGG11').cuda()]



all_accuracy_list=[]
all_test_accu_list=[]
for net in model_list2:
    for optimizer in ['SGD','Adam']:
        print("\n==============> Start Model")
        trial_list=[]
        criterion = nn.CrossEntropyLoss()
        optimizer=choose_optim(optimizer) #either SGD or Adam
        cur_model_accuracy_list=[]
        test_accu_avg=[]
        num_iterations=11

        for i in tqdm(np.arange(1,num_iterations,1)):
            train_loss_ls=[] #a list contains log train_loss
            train_accu_ls=[]
            test_loss_ls=[]
            test_accu_ls=[]
            end_epoch=8

            cur_training_best_accuracy_list=[0 for i in range(10)]
            for epoch in range(start_epoch, end_epoch):
                train_loss,train_accu=train()
                test_loss,test_accu=test()
                train_loss_ls.append(train_loss)
                train_accu_ls.append(train_accu)
                test_loss_ls.append(test_loss)
                test_accu_ls.append(test_accu)
                #separate classes accuracy
                sep_acc_ls=get_class_accuracy(testloader)
                cur_training_best_accuracy_list=list_update(cur_training_best_accuracy_list,sep_acc_ls)
            print('\n=======> Finished Training Trial '+str(i))
    #         plot(train_loss_ls,test_loss_ls,train_accu_ls,test_accu_ls)
            test_accu_avg.append(mean(test_accu_ls))
            cur_model_accuracy_list.append(cur_training_best_accuracy_list)
        all_accuracy_list.append(cur_model_accuracy_list)
        all_test_accu_list.append(test_accu_avg)
    
    
#     acc_class_df=pd.DataFrame(np.array(sep_accu_sum).reshape(10,-1),
#                           index=['plane','car','bird','cat',
#                                 'deer','dog','frog','horse',
#                                 'ship','truck'])
#     print("10 Separate Classes's Accuracy", acc_class_df)
    print("===============> End This Net")
all_accuracy_arr=np.array(all_accuracy_list)  
accu_df=pd.DataFrame(np.array(all_test_accu_list).reshape(num_iterations,-1),
                    index=['resnet3','resnet4(resnet18)','resnet5'])

==> Begin Building model..

==============> Start Model


  0%|                                                                                                                                                                                       | 0/10 [00:00<?, ?it/s]


Epoch: 0
Iterations: 12800/50048 | Train Loss: 2.233445949554443 | Train Acc: 19.109375
Iterations: 25600/50048 | Train Loss: 2.033342404961586 | Train Acc: 24.125
Iterations: 38400/50048 | Train Loss: 1.9122457540035247 | Train Acc: 28.075520833333332
Iterations: 50048/50048 | Train Loss: 1.8134496367495994 | Train Acc: 31.906
Iterations: 10000/10000 | Test Loss: 1.5030891942977904 | Test Acc: 44.41

Best Testing Accuracy: 44.41

Epoch: 1
Iterations: 12800/50048 | Train Loss: 1.3448491489887238 | Train Acc: 50.7421875
Iterations: 25600/50048 | Train Loss: 1.2883844783902167 | Train Acc: 52.92578125
Iterations: 38400/50048 | Train Loss: 1.2227239030599595 | Train Acc: 55.3984375
Iterations: 50048/50048 | Train Loss: 1.1837770883994334 | Train Acc: 57.032
Iterations: 10000/10000 | Test Loss: 1.1277410703897477 | Test Acc: 60.65

Best Testing Accuracy: 60.65

Epoch: 2
Iterations: 12800/50048 | Train Loss: 0.9741454368829727 | Train Acc: 65.6328125
Iterations: 25600/50048 | Train Loss: 0

 10%|█████████████████▏                                                                                                                                                          | 1/10 [08:05<1:12:49, 485.55s/it]


Epoch: 0
Iterations: 12800/50048 | Train Loss: 0.5206081387400627 | Train Acc: 82.3828125
Iterations: 25600/50048 | Train Loss: 0.5104806433618069 | Train Acc: 82.5
Iterations: 38400/50048 | Train Loss: 0.5146259216467539 | Train Acc: 82.31510416666667
Iterations: 50048/50048 | Train Loss: 0.5104801945979028 | Train Acc: 82.47
Iterations: 10000/10000 | Test Loss: 0.6757900664210319 | Test Acc: 77.14

Best Testing Accuracy: 79.57

Epoch: 1
Iterations: 12800/50048 | Train Loss: 0.4776300197839737 | Train Acc: 84.0078125
Iterations: 25600/50048 | Train Loss: 0.4834186652302742 | Train Acc: 83.53515625
Iterations: 38400/50048 | Train Loss: 0.48593352307875953 | Train Acc: 83.36979166666667
Iterations: 50048/50048 | Train Loss: 0.4852150417197391 | Train Acc: 83.378
Iterations: 10000/10000 | Test Loss: 0.8214424812793731 | Test Acc: 74.35

Best Testing Accuracy: 79.57

Epoch: 2
Iterations: 12800/50048 | Train Loss: 0.4782395425438881 | Train Acc: 83.46875
Iterations: 25600/50048 | Train Lo

...

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                 | 9/10 [1:32:46<11:05, 665.43s/it]


Epoch: 0
Iterations: 12800/50048 | Train Loss: 0.19954680114984513 | Train Acc: 93.0078125
Iterations: 25600/50048 | Train Loss: 0.20762152288109065 | Train Acc: 92.7421875
Iterations: 38400/50048 | Train Loss: 0.20691261117657025 | Train Acc: 92.765625
Iterations: 50048/50048 | Train Loss: 0.209178689514737 | Train Acc: 92.724
Iterations: 10000/10000 | Test Loss: 0.4685144065320492 | Test Acc: 87.37

Best Testing Accuracy: 88.41

Epoch: 1
Iterations: 12800/50048 | Train Loss: 0.21174527324736117 | Train Acc: 92.7421875
Iterations: 25600/50048 | Train Loss: 0.21170410715043544 | Train Acc: 92.75
Iterations: 38400/50048 | Train Loss: 0.21186379462480545 | Train Acc: 92.796875
Iterations: 50048/50048 | Train Loss: 0.2095132605803897 | Train Acc: 92.83
Iterations: 10000/10000 | Test Loss: 0.4433037668466568 | Test Acc: 87.8

Best Testing Accuracy: 88.41

Epoch: 2
Iterations: 12800/50048 | Train Loss: 0.1884395758062601 | Train Acc: 93.671875
Iterations: 25600/50048 | Train Loss: 0.194977

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [1:45:31<00:00, 695.43s/it]


===============> End This Net


ValueError: cannot reshape array of size 20 into shape (11,newaxis)

In [9]:
np.save('first_two_models.npy',np.array(all_accuracy_list))

In [ ]:
all_df=pd.DataFrame(all_accuracy_arr.reshape(num_iterations*2,-1),columns=['plane','car','bird','cat',
                                'deer','dog','frog','horse',
                                'ship','truck'])
all_df

In [9]:
torch.cuda.empty_cache()